In [2]:
import keras
import glob
import cv2
import os
import gc
import pickle
import numpy as np
import pandas as pd
import tqdm
from models.SegNet import *
from models.U_net import *
from models.DeeplabV3Plus import *
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras.metrics import MeanIoU
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import  compute_class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

## 載入照片與標籤

In [3]:
# 調整圖片大小，如果有需要
SIZE_X = 256 
SIZE_Y = 256
n_classes= 4  # 分割的類別數量=岩石分類+背景(1)=n_classes
random_number = 0
batch_size = 16
epochs_num = 3
model_name = f'random_{random_number}.h5'

In [4]:
# 捕捉訓練圖片信息，以列表形式保存
train_images = []

for img_path in tqdm.tqdm(glob.glob('../raw_data/core_image/*.png'), desc="正在載入照片"):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (SIZE_Y, SIZE_X))
    train_images.append(img)
    # 提取文件名並附加到列表中
    image_name = os.path.basename(img_path)

# 將列表轉換為陣列，以供機器學習處理        
train_images = np.array(train_images)

正在載入照片: 100%|██████████| 3943/3943 [01:35<00:00, 41.40it/s] 


In [5]:
# 捕捉遮罩/標籤信息，以列表形式保存
train_masks = [] 
for mask_path in tqdm.tqdm(glob.glob(f'../raw_data/{int(n_classes)-1}_label/*.png'), desc="正在載入標籤"):
    mask = cv2.imread(mask_path, 0)
    mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation=cv2.INTER_NEAREST)
    train_masks.append(mask)
        
# 將列表轉換為陣列，以供機器學習處理          
train_masks = np.array(train_masks)  # 這是載入照片及標籤的程式

正在載入標籤:   0%|          | 0/3943 [00:00<?, ?it/s]

正在載入標籤: 100%|██████████| 3943/3943 [00:05<00:00, 668.42it/s] 


In [6]:
print("train image shape: " + str(train_images.shape))
print("train mask shape: " + str(train_masks.shape))

train image shape: (3943, 256, 256, 3)
train mask shape: (3943, 256, 256)


## 訓練/測試集

In [7]:
###############################################
#Encode labels... but multi dim array so need to flatten, encode and reshape
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped.ravel())
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

print(train_masks_encoded_original_shape.shape)
np.unique(train_masks_encoded_original_shape)

(3943, 256, 256)


array([0, 1, 2, 3], dtype=int64)

In [8]:
#train_images = np.expand_dims(train_images, axis=3)
train_images_norm = train_images/255

train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)
###del
del train_masks_encoded_original_shape,train_masks_reshaped
gc.collect()

224

In [9]:
X_train, X_test, y_train, y_test= train_test_split(
    train_images_norm, 
    train_masks_input, 
    test_size=0.2, 
    random_state=random_number
)
# Print shapes of the sets
print("Training set shape: ", X_train.shape)
print("Validation set shape: ", X_test.shape)

Training set shape:  (3154, 256, 256, 3)
Validation set shape:  (789, 256, 256, 3)


In [10]:
del train_images_norm, train_masks_input
gc.collect()

0

In [11]:
print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled 

train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))

test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

Class values in the dataset are ...  [0 1 2 3]


In [12]:
class_weights =compute_class_weight(class_weight = "balanced",
                                        classes = np.unique(train_masks_reshaped_encoded),
                                        y = train_masks_reshaped_encoded )                                    
class_weights = class_weights.tolist()
class_weights

[1.0230623010003006, 0.7541217329575782, 1.6451587108611456, 0.91856678293266]

In [13]:
# 將 class_weights 寫入到.pkl檔案
with open(f'./random_number_{random_number}.pkl', 'wb') as f:
    pickle.dump(class_weights, f)

print("class_weights 已保存到 class_weights.pkl 檔案中。")

class_weights 已保存到 class_weights.pkl 檔案中。


---

In [14]:
# 从文件中加载 class_weights 对象
with open(f'./random_number_{random_number}.pkl', 'rb') as f:
    class_weights = pickle.load(f)
class_weights

[1.0230623010003006, 0.7541217329575782, 1.6451587108611456, 0.91856678293266]

In [18]:
#使用U-Net
# model =  U_NET(n_classes= n_classes, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=3)
# model.compile(optimizer='adam', 
#               loss= weightedLoss(keras.losses.categorical_crossentropy, class_weights), 
#               metrics=['accuracy'])
# model.summary()
#使用segNet
# model =  segnet(n_classes=4, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=3)
# model.compile(optimizer=SGD(learning_rate=0.001, 
#                                 momentum=0.9, 
#                                 decay=0.0005, 
#                                 nesterov=False), 
#                                 loss='categorical_crossentropy', 
#                                 metrics=['accuracy']) # type: ignore
#使用DeeplabV3Plus
model = DeeplabV3Plus(image_size=256, num_classes=n_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

94765736/94765736 [==============================] - 4s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 256, 256, 3)  0          ['input_1[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 tf.nn.bias_add (TFOpLambda)    (None, 256, 256, 3)  0           ['tf.__operators__.getitem[0][0]'
                             

In [19]:
learning_rate_function = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.6,
                                            min_lr=0.00001)
earlystop = EarlyStopping(monitor='val_loss',
                          mode='min',
                          #min_delta=0.001,
                          patience=60,
                          verbose=1)
modelcheck = ModelCheckpoint(model_name,
                             monitor='val_loss',
                             mode='min',
                             verbose=1,
                             save_best_only=True)

In [20]:
history = model.fit(X_train, 
                 y_train_cat, 
                 epochs= epochs_num, 
                 batch_size= batch_size, 
                 validation_data= (X_test, y_test_cat), 
                 verbose=1, 
                 shuffle=False,
                 callbacks=[learning_rate_function, earlystop, modelcheck])

Epoch 1/3


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/conv2d_9/Conv2D/Conv2DBackpropInput' defined at (most recent call last):
    File "c:\Users\User\anaconda3\envs\well_work\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\User\anaconda3\envs\well_work\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\User\anaconda3\envs\well_work\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\User\anaconda3\envs\well_work\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\Users\User\anaconda3\envs\well_work\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "c:\Users\User\anaconda3\envs\well_work\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_fore

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = range(1, len(history.history['loss']) + 1)
plt.plot(N, history.history["loss"], label="train_loss")
plt.plot(N, history.history["val_loss"], label="val_loss")
plt.plot(N, history.history["accuracy"], label="train_acc")
plt.plot(N, history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig(f'./U_net_Loss&Accuracy{random_number}.png')

--- 

## 載入模型

In [ ]:
model.load_weights(f'./{model_name}')
############################################################

In [ ]:
#Evaluate the model
# evaluate model
_, acc = model.evaluate(X_train, y_train_cat)
print("Accuracy is = ", (acc * 100.0), "%")

## 驗證集預測結果

In [ ]:
##################################
#IOU
val_y_pred=model.predict(X_test)
val_y_pred_argmax=np.argmax(val_y_pred, axis=3)

In [ ]:
#Using built in keras function

IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], val_y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
#Using built in keras function
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], val_y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())


#To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
class0_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3]+ values[1,0]+ values[2,0]+ values[3,0])
class1_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3]+ values[0,1]+ values[2,1]+ values[3,1])
class2_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3]+ values[0,2]+ values[1,2]+ values[3,2])
class3_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2]+ values[0,3]+ values[1,3]+ values[2,3])
#class4_IoU = values[4,4]/(values[4,4] + values[4,0] + values[4,1] + values[4,2]+ values[4,3]+ values[0,4]+ values[1,4]+ values[2,4]+ values[3,4])


print("IoU for class0 is: ", class0_IoU)
print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)
#print("IoU for class4 is: ", class4_IoU)

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,name:str, normalize=False, title="Confusion Matrix", cmap=plt.cm.Blues):
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    plt.grid(False)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(f'confusion_matrix_{name}.png')

In [ ]:
values_cm = int_array = np.array(IOU_keras.get_weights()).astype(int)
values_cm  = values_cm .reshape(n_classes, n_classes)
target_names = ["background","mud","sand","gravel"]
plot_confusion_matrix(values_cm , target_names,f'{model_name}')
classification_report_test = classification_report(y_test[:,:,:,0].ravel(),
                                                   val_y_pred_argmax.ravel(),
                                                    target_names=target_names, 
                                                    digits = 4)
print(classification_report_test)#會輸出precision、recall與f1-score。

### Testing dataset 預測結果

In [ ]:
# 捕捉訓練圖片信息，以列表形式保存
test_images = []


for img_path in glob.glob('../raw_data/testing_dataset/core_image/*.png'):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (SIZE_Y, SIZE_X))
    test_images.append(img)
    # 提取文件名並附加到列表中
    image_name = os.path.basename(img_path)

# 將列表轉換為陣列，以供機器學習處理        
test_images = np.array(test_images)

# 捕捉遮罩/標籤信息，以列表形式保存
test_masks = [] 
name_list = []

for mask_path in glob.glob(f'../raw_data/testing_dataset/{int(n_classes)-1}_label/*.png'):
    # 获取文件名并添加到列表中
    name = os.path.basename(mask_path)
    name_list.append(name)
    mask = cv2.imread(mask_path, 0)
    mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation=cv2.INTER_NEAREST)
    test_masks.append(mask)
        
# 將列表轉換為陣列，以供機器學習處理          
test_masks = np.array(test_masks)  # 這是載入照片及標籤的程式

In [ ]:
print("test image shape: " + str(test_images.shape))
print("test mask shape: " + str(test_masks.shape))

In [ ]:
###############################################
#Encode labels... but multi dim array so need to flatten, encode and reshape
labelencoder = LabelEncoder()
n, h, w = test_masks.shape
test_masks_reshaped = test_masks.reshape(-1,1)
test_masks_reshaped_encoded = labelencoder.fit_transform(test_masks_reshaped.ravel())
test_masks_encoded_original_shape = test_masks_reshaped_encoded.reshape(n, h, w)

print(test_masks_encoded_original_shape.shape)
np.unique(test_masks_encoded_original_shape)

In [ ]:
test_images_norm = test_images/255

test_masks_input = np.expand_dims(test_masks_encoded_original_shape, axis=3)

In [ ]:
##################################
#IOU
y_pred=model.predict(test_images_norm)
y_pred_argmax=np.argmax(y_pred, axis=3)

In [ ]:
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(test_masks_input[:,:,:,0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(test_masks.ravel(), y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())


#To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
class0_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3]+ values[1,0]+ values[2,0]+ values[3,0])
class1_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3]+ values[0,1]+ values[2,1]+ values[3,1])
class2_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3]+ values[0,2]+ values[1,2]+ values[3,2])
class3_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2]+ values[0,3]+ values[1,3]+ values[2,3])
#class4_IoU = values[4,4]/(values[4,4] + values[4,0] + values[4,1] + values[4,2]+ values[4,3]+ values[0,4]+ values[1,4]+ values[2,4]+ values[3,4])


print("IoU for class0 is: ", class0_IoU)
print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)
#print("IoU for class4 is: ", class4_IoU)

In [ ]:
values_cm = int_array = np.array(IOU_keras.get_weights()).astype(int)
values_cm  = values_cm .reshape(n_classes, n_classes)
target_names = ["background","mud","sand","gravel"]
plot_confusion_matrix(values_cm , target_names,f'testing_{model_name}')
classification_report_test = classification_report(test_masks.ravel(),
                                                   y_pred_argmax.ravel(),
                                                    target_names=target_names, 
                                                    digits = 4)
print(classification_report_test)#會輸出precision、recall與f1-score。

----

## 寫入excel

In [ ]:
val_report = classification_report(y_test[:,:,:,0].ravel(),
                                val_y_pred_argmax.ravel(), 
                                target_names=target_names, 
                                digits = 4,
                                output_dict=True)
val_df = pd.DataFrame(val_report).transpose()
val_df

In [ ]:
new_row_data = {}
for col in val_df.columns:
    new_row_data[col] = 'validation'

new_row = pd.DataFrame(new_row_data, index=[0])
new_row

In [ ]:
val_df = pd.concat([new_row, val_df], ignore_index=False, axis=0)
val_df

In [ ]:
test_report = classification_report(test_masks.ravel(),
                                   y_pred_argmax.ravel(),
                                    target_names=target_names, 
                                    digits = 4,
                                    output_dict=True)

In [ ]:
new_row_data = {}
for col in val_df.columns:
    new_row_data[col] = 'testing'

new_row = pd.DataFrame(new_row_data, index=[0])

In [ ]:
test_df = pd.DataFrame(test_report).transpose()

In [ ]:
test_df = pd.concat([new_row, test_df], ignore_index=False)
test_df

In [ ]:
df = pd.concat([val_df, test_df], ignore_index=False)
df

In [ ]:
# 检查文件是否存在
if not os.path.exists('./segnet_result.xlsx'):
    # 将 DataFrame 保存为 Excel 文件
    df.to_excel('./segnet_result.xlsx', sheet_name=f'random_{random_number}')
else:
    # 如果文件已存在，则不执行任何操作
    pass

In [ ]:
with pd.ExcelWriter('./segnet_result.xlsx', engine='openpyxl', mode='a') as writer:
    # 寫入 DataFrame 到現有的 Excel 檔案的指定工作表中
    df.to_excel(writer, index=True, sheet_name=f'random_{random_number}')